In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,InputLayer
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('all') 

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/MyDrive/CS6890Project/data/training.1600000.processed.noemoticon.csv"

In [ ]:
file = open(data_path, "r")
for line in file:
    print(line)
    break
file.close()

"0","1467810369","Mon Apr 06 22:19:45 PDT 2009","NO_QUERY","_TheSpecialOne_","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"



In [ ]:
data = pd.read_csv(data_path, header=None, encoding = "ISO-8859-1")
data.columns = ["target", "id", "date", "flag", "user", "text"]

In [ ]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.dtypes

target     int64
id         int64
date      object
flag      object
user      object
text      object
dtype: object

In [ ]:
X=np.array(data['text'])
Y=np.array(data['target'])

# Set Y=1 for Positive Tweets
Y[Y==4]=1

In [ ]:
index = 123  # index in range [0,1599999]

print(X[index])

I am in pain. My back and sides hurt. Not to mention crying is made of fail. 


In [ ]:
def tokenize(X):
  """
  Tokenize the data using nltk
  """

  treebank = nltk.tokenize.TreebankWordTokenizer()
  X_tokenized=[treebank.tokenize(sentence) for sentence in X]  
  return X_tokenized


def remove_stopwords(X):
  """
  Remove Stopwords using nltk
  """

  stopwords=nltk.corpus.stopwords.words('english') + ['@']
  X_nostopwords = []

  for sentence in X:

    temp = [word for word in sentence if not word in stopwords]
    X_nostopwords.append(temp) 

  return X_nostopwords

#tokenize

In [ ]:

tokenizer = Tokenizer(filters='@')
tokenizer.fit_on_texts(X)

print('No. of distinct tokens = '+str(len(tokenizer.word_index)))

No. of distinct tokens = 1185412


In [ ]:
max_vocab=50000

In [ ]:
tokenizer = Tokenizer(num_words=max_vocab,filters='@')
tokenizer.fit_on_texts(X)

In [ ]:
X_vectorized=tokenizer.texts_to_sequences(X)

In [ ]:
sum=0
for sentence in X_vectorized:
  sum+=len(sentence)

print('Average length of tweets = '+str(sum/len(X_vectorized)))

Average length of tweets = 11.967963125


#padding

In [ ]:
max_length=20
X_pad = keras.preprocessing.sequence.pad_sequences(X_vectorized,max_length,padding='post',
                                                   truncating='post')

print(X_pad.shape)

(1600000, 20)


In [ ]:
index  = 123  # index in range [0,1599999]
X_tokenized = tokenize(X)

print('\nOriginal :')
print(X[index])
print('\nTokenized :')
print(X_tokenized[index])
print('\nVectorized :')
print(X_vectorized[index])
print('\nPadded :')
print(X_pad[index])




Original :
I am in pain. My back and sides hurt. Not to mention crying is made of fail. 

Tokenized :
['I', 'am', 'in', 'pain.', 'My', 'back', 'and', 'sides', 'hurt.', 'Not', 'to', 'mention', 'crying', 'is', 'made', 'of', 'fail', '.']

Vectorized :
[1, 57, 10, 2588, 5, 48, 6, 8826, 2898, 25, 2, 1418, 1086, 8, 187, 12, 2288]

Padded :
[   1   57   10 2588    5   48    6 8826 2898   25    2 1418 1086    8
  187   12 2288    0    0    0]


In [25]:
np.random.seed(123)

model_1 = keras.models.Sequential(name='model1')

model_1.add(InputLayer(input_shape=(20,),name = 'Integer_Encoding'))
model_1.add(Embedding(max_vocab, 100,input_length=20,mask_zero=True,
                      name='100D_Encoding',trainable=False))
model_1.add(Bidirectional(LSTM(64,name='LSTM'),name='Bidirectional_RNN'))
model_1.add(Dropout(0.5,name='Regularizer'))
model_1.add(Dense(1, activation='sigmoid',name='Sigmoid_Classifier'))

In [26]:
import tensorflow as tf 
from tensorflow.keras.utils import plot_model

model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',
                metrics=['accuracy'])
model_1

In [27]:
model_1.summary()
# tf.keras.utils.plot_model(model_1,to_file='model_1.png')

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 100D_Encoding (Embedding)   (None, 20, 100)           5000000   
                                                                 
 Bidirectional_RNN (Bidirect  (None, 128)              84480     
 ional)                                                          
                                                                 
 Regularizer (Dropout)       (None, 128)               0         
                                                                 
 Sigmoid_Classifier (Dense)  (None, 1)                 129       
                                                                 
Total params: 5,084,609
Trainable params: 84,609
Non-trainable params: 5,000,000
_________________________________________________________________


In [28]:
np.random.seed(123)

X_train, X_test, Y_train, Y_test = train_test_split (X_pad,Y.reshape(Y.shape[0],1),test_size=0.20)

# X_train = np.asarray(X_train)
# X_test = np.asarray(X_test)
# Y_train = np.asarray(Y_train)
# Y_test = np.asarray(Y_test)

print(len(X_train))
print(len(X_test))
print(len(Y_train))
print(len(Y_test))

1280000
320000
1280000
320000


In [ ]:
# X_train = np.array(X_train,dtype=object)
# X_test = np.array(X_test,dtype=object)
# Y_train = np.array(Y_train,dtype=object)
# Y_test = np.array(Y_test,dtype=object)


In [29]:
history_model_1 = model_1.fit(X_train,Y_train,batch_size=1024,epochs=15,validation_split=0.20)

Epoch 1/15
1000/1000 [==============================] - 372s 365ms/step - loss: 0.6457 - accuracy: 0.6252 - val_loss: 0.6237 - val_accuracy: 0.6473
Epoch 2/15
1000/1000 [==============================] - 371s 371ms/step - loss: 0.6038 - accuracy: 0.6682 - val_loss: 0.5875 - val_accuracy: 0.6839
Epoch 3/15
1000/1000 [==============================] - 371s 371ms/step - loss: 0.5712 - accuracy: 0.6983 - val_loss: 0.5518 - val_accuracy: 0.7143
Epoch 4/15
1000/1000 [==============================] - 373s 373ms/step - loss: 0.5502 - accuracy: 0.7164 - val_loss: 0.5357 - val_accuracy: 0.7287
Epoch 5/15
1000/1000 [==============================] - 378s 378ms/step - loss: 0.5364 - accuracy: 0.7274 - val_loss: 0.5270 - val_accuracy: 0.7339
Epoch 6/15
1000/1000 [==============================] - 383s 383ms/step - loss: 0.5259 - accuracy: 0.7356 - val_loss: 0.5150 - val_accuracy: 0.7426
Epoch 7/15
1000/1000 [==============================] - 380s 380ms/step - loss: 0.5182 - accuracy: 0.7415 - val_

In [30]:
model_1.evaluate(X_test,Y_test)

10000/10000 [==============================] - 117s 11ms/step - loss: 0.4864 - accuracy: 0.7627


[0.48643872141838074, 0.7627249956130981]

In [ ]:
from keras.layers import *
from keras.models import *
from keras import backend as K

In [ ]:
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences

        super(attention,self).__init__()

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1), initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1))
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1))
        super(attention,self).build(input_shape)


    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:

            return output
        return K.sum(output, axis=1)


In [ ]:
def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),initializer="zeros")

In [ ]:
def call(self, x):
    e = K.tanh(K.dot(x,self.W)+self.b)
    a = K.softmax(e, axis=1)
    output = x*a


In [32]:
np.random.seed(123)

model_2 = keras.models.Sequential(name='model2')

model_2.add(InputLayer(input_shape=(20,),name = 'Integer_Encoding'))
model_2.add(Embedding(max_vocab, 100,input_length=20,mask_zero=True,
                      name='100D_Encoding',trainable=False))
model_2.add(Bidirectional(LSTM(64,name='LSTM'),name='Bidirectional_RNN'))
# model_2.add(attention(return_sequences=True))
model_2.add(Dropout(0.5,name='Regularizer'))
model_2.add(Dense(1, activation='sigmoid',name='Sigmoid_Classifier'))

In [33]:
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',
                metrics=['accuracy'])
model_2


In [34]:
model_2.summary()

Model: "model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 100D_Encoding (Embedding)   (None, 20, 100)           5000000   
                                                                 
 Bidirectional_RNN (Bidirect  (None, 128)              84480     
 ional)                                                          
                                                                 
 Regularizer (Dropout)       (None, 128)               0         
                                                                 
 Sigmoid_Classifier (Dense)  (None, 1)                 129       
                                                                 
Total params: 5,084,609
Trainable params: 84,609
Non-trainable params: 5,000,000
_________________________________________________________________


In [35]:
np.random.seed(123)

X_train, X_test, Y_train, Y_test = train_test_split (X_pad,Y.reshape(Y.shape[0],1),test_size=0.20)

# X_train = np.asarray(X_train)
# X_test = np.asarray(X_test)
# Y_train = np.asarray(Y_train)
# Y_test = np.asarray(Y_test)

print(len(X_train))
print(len(X_test))
print(len(Y_train))
print(len(Y_test))

1280000
320000
1280000
320000


In [36]:
history_model_2 = model_2.fit(X_train,Y_train,batch_size=128,epochs=1,validation_split=0.20)

8000/8000 [==============================] - 587s 72ms/step - loss: 0.6167 - accuracy: 0.6545 - val_loss: 0.5855 - val_accuracy: 0.6813


#Word Embeddings

In [ ]:
def lemmatize(X):
  
  lemmatizer = nltk.stem.WordNetLemmatizer()
  
  X_pos = []
  X_lemmatized = []

  for sentence in X :

    temp = nltk.pos_tag(sentence)
    X_pos.append(temp)  

  for sentence in X_pos :

    temp = [ lemmatizer.lemmatize(remove_stopwords(word)) for word in X_pad]
    X_lemmatized.append(temp)  

  return X_lemmatized    

In [ ]:
X_tokenized = tokenize (X)

X_without_stopwords = remove_stopwords ( X_tokenized )

X_lemmatized = lemmatize ( X_without_stopwords )

X_clean = []

for sentence in X_lemmatized:

  temp = " ".join(sentence)
  X_clean.append(temp)

TypeError: ignored